<a href="https://colab.research.google.com/github/Riccardolotorto/pythonGEOPANDASnuovo/blob/main/GeoPandas_distance_DataFrame_to_GeoDataFrame_negazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas contextily

In [2]:
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sosta_turistici.csv", sep = ";")

1. trasformare il dataframe di pandas in un geodataframe di geopandas

In [6]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df["LONG_X_4326"], df["LAT_Y_4326"])]
df = df.drop(['LONG_X_4326', 'LAT_Y_4326'], axis=1)
parcheggi = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

In [ ]:
parcheggi.head()

2. mostrare il geodatframe su una mappa di contextily

In [8]:
parcheggi3857 = parcheggi.to_crs(3857)

In [ ]:
ax = parcheggi3857.plot(figsize = (17, 12), markersize = 10, color = 'red')
ctx.add_basemap(ax)

3. caricare il file dei quartieri di milano

In [11]:
quartieri = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/ds964_nil_wm.zip")

4. mostrare i quartieri su una mappa di contextily

In [12]:
quartieri3857 = quartieri.to_crs(3857)

In [ ]:
ax = quartieri3857.plot(figsize = (17, 12), edgecolor = "k", facecolor = 'none')
ctx.add_basemap(ax)

5. salvare in un nuovo geodataframe i quartieri che contengono parcheggi

In [ ]:
quartieriParcheggi = quartieri3857[quartieri3857.intersects(parcheggi3857.unary_union)]
quartieriParcheggi

6. mostrare sulla mappa di contextily i quartieriParcheggi e i parcheggi

In [ ]:
ax = quartieriParcheggi.plot(figsize = (17, 12), edgecolor = "k", facecolor = 'none')
parcheggi3857.plot(ax = ax, markersize = 10, color='red')
ctx.add_basemap(ax)

7. mostrare solo i parcheggi vicino ad una metropolitana

In [19]:
metropolitane = ['M1', 'M2', 'M3', 'M5']

In [20]:
parcheggiMetropolitana = parcheggi3857[parcheggi3857.localizzaz.str.contains('|'.join(metropolitane))]

In [ ]:
ax = parcheggiMetropolitana.plot(figsize = (17, 12), markersize = 10, color = 'red')
ctx.add_basemap(ax)

8. aggiumgere al dataset dei parcheggi i quartieri

In [ ]:
joined = gpd.sjoin(parcheggi3857, quartieriParcheggi, predicate = "intersects", how ='left')
joined

9. contare il numero di parcheggi per ogni quartiere

In [ ]:
numParPerQuartiere = joined.groupby("NIL")[['localizzaz']].count().reset_index()
numParPerQuartiere

10. mostrare i quartieri di milano colorati diversamente in base al numero di parcheggi presenti

In [ ]:
finale = quartieriParcheggi.merge(numParPerQuartiere, on='NIL')
ax = finale.plot(figsize = (17, 12), column = 'localizzaz', legend = True, cmap = 'Reds')
ctx.add_basemap(ax)

11. salvare i quartieri di milano che non contengono parcheggi

In [25]:
quartieriNOParcheggi = quartieri3857[~quartieri3857.intersects(parcheggi3857.unary_union)]

In [ ]:
ax = quartieriNOParcheggi.plot(figsize = (17, 12), edgecolor = "k", facecolor = 'none')
ctx.add_basemap(ax)

12. mostrare solo il confine di milano (usando però il dataset dei quartiere)

In [ ]:
from shapely.ops import cascaded_union
milanoOggetto = cascaded_union(quartieri3857.geometry)
milano = gpd.GeoSeries(milanoOggetto)

In [ ]:
ax = milano.plot(figsize = (17, 12), edgecolor = 'k', facecolor = 'none', linewidth = 2)
ctx.add_basemap(ax)

13. visualizzare il quartiere con più parcheggi (elenco e mappa)

In [ ]:
quartiereMaggiore = finale[finale.localizzaz == finale.localizzaz.max()]
quartiereMaggiore

In [ ]:
ax = quartiereMaggiore.plot(figsize = (17, 12), edgecolor = 'k', facecolor = 'none')
ctx.add_basemap(ax)

14. analogo all'esercizio precedente, mostrare anche i parcheggi all'interno di esso

In [32]:
parcheggiQuartiereMaggiore = parcheggi3857[parcheggi3857.within(quartiereMaggiore.geometry.item())]

In [ ]:
ax = quartiereMaggiore.plot(figsize = (17, 12), edgecolor = 'k', facecolor = 'none')
parcheggiQuartiereMaggiore.plot(ax = ax, color = 'red', markersize = 15)
ctx.add_basemap(ax)

15. far inserire dall'utente un punto con coordinate geografiche e mostrare con elenco e mappa i parcheggi a meno di 500 metri di distanza

In [ ]:
from shapely.geometry import Point
longitudine = float(input("inserisci longitudine "))
latitudine = float(input("inserisci latitudine "))
punto = gpd.GeoSeries([Point(longitudine, latitudine)], crs = 4326).to_crs(3857)
m500 = parcheggi3857[parcheggi3857.distance(punto.geometry.item()) < 500]
m500

In [ ]:
ax = m500.plot(figsize = (17, 12), markersize = 40, color = 'red')
punto.plot(ax = ax, markersize = 15, color = 'blue')
ctx.add_basemap(ax)